In [3]:
%load_ext autoreload

%autoreload 2

In [227]:
from icd9cms import search
import os
import numpy as np
import pandas as pd
from collections import defaultdict
import json
from itertools import chain
import ipywidgets as widgets
from IPython.display import display
import re
from collections import Counter
import difflib
from differ import html_diffs, align_seqs, sentencize, \
    diff_ratio, show_diffs, compute_sequential_diff_metrics, \
    split_notes, compute_avgs
from datasets import list_metrics, load_metric
import seaborn as sns
from glob import glob
from IPython.display import clear_output

In [17]:
os.path.exists('../data/mimic_3_all_notes_with_20_occurrences_of_primary_condition.csv')

True

In [7]:
codes = pd.read_csv('../data/primary_codes.csv')
sorted_primary_codes_df =codes['icd9_code'].value_counts().rename_axis('code').reset_index()
sorted_primary_codes_df['long_title'] = sorted_primary_codes_df['code'].apply(lambda c: search(c).long_desc or search(c).short_desc)
sorted_primary_codes_df.columns = ['code', 'count', 'long_title']
sorted_primary_codes_df = sorted_primary_codes_df[~sorted_primary_codes_df.code.str.startswith('V3')]

In [8]:
sorted_primary_codes_df[sorted_primary_codes_df.code.str.startswith('41401')]

,code,count,long_title
1,41401,3498,Coronary atherosclerosis of native coronary ar...


In [9]:
sorted_primary_codes_df[sorted_primary_codes_df.code.str.startswith('41')].head(5)

,code,count,long_title
1,41401,3498,Coronary atherosclerosis of native coronary ar...
4,41071,1751,"Subendocardial infarction, initial episode of ..."
15,41041,482,Acute myocardial infarction of other inferior ...
16,41011,482,Acute myocardial infarction of other anterior ...
20,41519,343,Other pulmonary embolism and infarction


In [10]:
sorted_primary_codes_df[sorted_primary_codes_df.code.str.startswith('434')].long_title.tolist()

['Cerebral embolism with cerebral infarction',
 'Cerebral artery occlusion, unspecified with cerebral infarction',
 'Cerebral thrombosis with cerebral infarction',
 'Cerebral artery occlusion, unspecified without mention of cerebral infarction',
 'Cerebral embolism without mention of cerebral infarction']

### 41401 Notes

In [30]:
notes = pd.read_csv('../data/41401_notes.csv')

In [31]:
discharge_notes = notes[(notes.category == 'Discharge summary') & (notes.description != 'Addendum')]

In [33]:
care_notes_df, discharge_notes_df = split_notes(notes)

In [34]:
care_notes_df['cat_descs'] = care_notes_df.apply(lambda r: [(f'{c.strip()}:{d}') for c,d in zip(r.categories, r.descriptions)], axis=1)

In [23]:
results_df = compute_diff(stroke_notes_subset)

2021-01-19 12:49:06,517: Removing /Users/tom/.cache/huggingface/metrics/bert_score/default/default_experiment-1-0.arrow
2021-01-19 12:49:06,524: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow


In [386]:
results_df = pd.concat([rougeL, results_df.loc[:, [c for c in results_df.columns if 'rougeL' in c]]], axis=1)

In [399]:
scores = results_df.loc[:, ['category', 'description', 'avg_diff_ratio', 'rougeL_recall_med', 'rougeL_recall_iqr', 'rougeL_precision_med', 'rougeL_precision_iqr', 
                            'bert-score_recall_med', 'bert-score_recall_iqr', 'bert-score_precision_med', 'bert-score_precision_iqr']]

In [400]:
scores.sort_values(['rougeL_recall_med', 'rougeL_precision_med'], ascending=[False, True])

,category,description,avg_diff_ratio,rougeL_recall_med,rougeL_recall_iqr,rougeL_precision_med,rougeL_precision_iqr,bert-score_recall_med,bert-score_recall_iqr,bert-score_precision_med,bert-score_precision_iqr
2,Nursing,Nursing Transfer Note,0.872569,0.997854,0.000000,0.621658,0.000000,0.982755,0.000000,0.744808,0.000000
3,Nutrition,Clinical Nutrition Note,0.154612,0.921967,0.323136,0.523822,0.138380,0.916548,0.203272,0.601892,0.248073
4,Physician,Intensivist Note,0.234087,0.651907,0.127212,0.692593,0.118770,0.857477,0.060351,0.853575,0.046447
1,Nursing,Nursing Progress Note,0.440298,0.632599,0.675619,0.666618,0.682953,0.701072,0.424521,0.730515,0.501814
5,Radiology,CHEST (PORTABLE AP),0.088830,0.417902,0.021350,0.540940,0.270825,0.791767,0.009032,0.827279,0.074302
6,Radiology,CT HEAD W/O CONTRAST,0.088893,0.411212,0.231364,0.399956,0.201956,0.755250,0.107351,0.772630,0.083226
7,Social Work,Social Work Progress Note,0.073434,0.383871,0.000000,0.162568,0.000000,0.702947,0.000000,0.552549,0.000000
0,ECG,Report,0.186325,0.331579,0.167044,0.231640,0.159380,0.569639,0.144755,0.428310,0.057963


In [402]:
scores.sort_values(['bert-score_recall_med', 'bert-score_precision_med'], ascending=[False, True])

,category,description,avg_diff_ratio,rougeL_recall_med,rougeL_recall_iqr,rougeL_precision_med,rougeL_precision_iqr,bert-score_recall_med,bert-score_recall_iqr,bert-score_precision_med,bert-score_precision_iqr
2,Nursing,Nursing Transfer Note,0.872569,0.997854,0.000000,0.621658,0.000000,0.982755,0.000000,0.744808,0.000000
3,Nutrition,Clinical Nutrition Note,0.154612,0.921967,0.323136,0.523822,0.138380,0.916548,0.203272,0.601892,0.248073
4,Physician,Intensivist Note,0.234087,0.651907,0.127212,0.692593,0.118770,0.857477,0.060351,0.853575,0.046447
5,Radiology,CHEST (PORTABLE AP),0.088830,0.417902,0.021350,0.540940,0.270825,0.791767,0.009032,0.827279,0.074302
6,Radiology,CT HEAD W/O CONTRAST,0.088893,0.411212,0.231364,0.399956,0.201956,0.755250,0.107351,0.772630,0.083226
7,Social Work,Social Work Progress Note,0.073434,0.383871,0.000000,0.162568,0.000000,0.702947,0.000000,0.552549,0.000000
1,Nursing,Nursing Progress Note,0.440298,0.632599,0.675619,0.666618,0.682953,0.701072,0.424521,0.730515,0.501814
0,ECG,Report,0.186325,0.331579,0.167044,0.231640,0.159380,0.569639,0.144755,0.428310,0.057963


In [ ]:
# sort and re-rank, set of documents, extract phenotypes to test how excluding 
# the set of redundant document effects phenotype extraction??

In [ ]:
# plots between max / min... as things progress... - heavy bias between how the clinician records the notes..

In [479]:
avgs = compute_avgs(more_notes_results_df)

In [495]:
avgs.loc[:, ['cat_desc', 'micro_avg', 'rg_rec_avg', 'rg_prec_avg', 'bs_rec_avg', 'bs_prec_avg']]

,cat_desc,micro_avg,rg_rec_avg,rg_prec_avg,bs_rec_avg,bs_prec_avg
0,Nursing:Nursing Progress Note,0.227994,0.345551,0.348121,0.496873,0.494258
1,Physician :Intensivist Note,0.169265,0.594225,0.597550,0.856726,0.858064
2,Radiology:CT HEAD W/O CONTRAST,0.084276,0.400569,0.471806,0.732597,0.739952
3,Nursing:Nursing Transfer Note,0.421328,0.725352,0.545839,0.701177,0.572983
4,Nutrition:Clinical Nutrition Note,0.278534,0.723751,0.591569,0.789677,0.708747


### Compare Discharge Summary to last note types via Summarisation Metrics
- recall will likely be low, but any non-zero ness is potentially interesting
- do neural methods work at all here??

In [ ]:
# only take last note...? Or take all notes of each category
#

In [502]:
rouge_metric.compute(rouge_types=['rougeL'], use_agregator=False)['rougeL']

2021-01-15 00:19:26,721: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow


[Score(precision=0.3333333333333333, recall=1.0, fmeasure=0.5),
 Score(precision=0.3333333333333333, recall=1.0, fmeasure=0.5)]

In [11]:
def calc_diff_metrics(ds: str, prev_notes: dict):
    # calc diff between 
    rouge_metric = load_metric('rouge')
    bert_metric = load_metric('bertscore')
    output = defaultdict(list)
    for k, note_text in prev_notes.items():
        ratio, std_dev, max_ratio, min_ratio, redundant_toks_len, total_toks = diff_ratio(ds, note_text)
        output['ratio'].append(ratio)
        output['cat_desc'].append(':'.join(k))
        rouge_metric.add(prediction=ds, reference=note_text)
        bert_metric.add(prediction=ds, reference=note_text)
#     bert_scores = bert_metric.compute(lang='en', rescale_with_baseline=True, model_type='xlnet-base-cased')
#     output['bs_prec'].extend(bert_scores.precision)
#     output['bs_rec'].extend(bert_scores.recall)
    rougeL = rouge_metric.compute(rouge_types=['rougeL'], use_agregator=False)['rougeL']
    output['rg_prec'].extend([s.precision for s in rougeL])
    output['rg_rec'].extend([s.recall for s in rougeL])
    return pd.DataFrame(output)
    
def calc_discharge_summary_redundancy(notes: pd.DataFrame):
    summaries = notes[notes.category == 'Discharge summary'].loc[:, ['hadm_id', 'text']]
    ds_diffs = []
    for r in summaries.itertuples():
        hadm_notes = notes[(notes.hadm_id == r.hadm_id) & (notes.category != 'Discharge summary')]
        last_notes_per_type = {}
        for k, df_g in hadm_notes.groupby(['category', 'description']):
            last_notes_per_type[':'.join(k)] = df_g.text.iloc[-1]
        ds_diffs.append(calc_diff_metrics(r.text, last_notes_per_type))
    return pd.concat(ds_diffs)

In [518]:
calc_discharge_summary_redundancy(stroke_notes_subset)

2021-01-15 00:24:18,249: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow
2021-01-15 00:24:21,168: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow


In [260]:
all_data = pd.read_csv('../data/mimic_3_all_notes_with_20_occurrences_of_primary_condition.csv')

/home/thomass/anaconda3/envs/transformers/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
len(all_data.hadm_id.unique())

41463

In [4]:
all_data = all_data.loc[:, ['row_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime', 'storetime', 'category', 'description', 'cgid', 'iserror', 'text', 'icd9_code']]

In [238]:
all_data.to_csv('../data/mimic_3_all_notes_with_20_occurrences_of_primary_condition.csv')

In [7]:
block_size = 500
df_splits = []
adm_ids = all_data.hadm_id.unique()
for block_num in range(int(np.ceil(len(adm_ids) / block_size))):
    block_start = block_num * block_size
    block_end = (block_num+1) * block_size 
    ids = adm_ids[block_num * block_size:block_end if block_end < len(adm_ids) else len(adm_ids)]
    df_splits.append(all_data[all_data.hadm_id.isin(ids)])

In [8]:
from multiprocessing import Pool
import torch

In [9]:
torch.multiprocessing.set_start_method('spawn')

In [ ]:
with Pool(3) as p:
    ms_res = p.map(compute_sequential_diff_metrics, [df_splits[0], df_splits[1], df_splits[2]])

In [38]:
print('Token level / global note redundancy with concurrent notes:')
print(f'Overall Average Internote type redundnacy:{results_df.avg_diff_ratio.sum() / results_df.avg_diff_ratio.shape[0]}')

Token level / global note redundancy with concurrent notes:


NameError: name 'results_df' is not defined

### 43411 Note Analysis

In [59]:
stroke_notes = pd.read_csv('../data/43411_notes.csv')

In [62]:
len(stroke_notes.hadm_id.unique())

337

In [63]:
stroke_notes.shape

(8213, 16)

In [251]:
stroke_subset = stroke_notes[stroke_notes.hadm_id.isin(stroke_notes.hadm_id.unique()[0:2])]

In [252]:
stroke_subset.shape

(159, 16)

In [258]:
stroke_subset.to_csv('../data/stroke_subset.csv')

In [65]:
avgs = compute_avgs(df)

In [66]:
avgs.columns

Index(['cat_desc', 'redundant_toks', 'total_toks', 'avg_txt_len', 'macro_avg',
       'micro_avg', 'num_instances', 'rg_rec_avg', 'rg_rec_med', 'rg_rec_iqr',
       'rg_prec_avg', 'rg_prec_med', 'rg_prec_iqr', 'bs_rec_avg', 'bs_rec_med',
       'bs_rec_iqr', 'bs_prec_avg', 'bs_prec_med', 'bs_prec_iqr'],
      dtype='object')

In [71]:
avgs_condensed = avgs.loc[:, ['cat_desc', 'num_instances', 'macro_avg', 'rg_rec_med', 'rg_prec_med', 'bs_rec_med', 'bs_prec_med']]

In [73]:
avgs_condensed.sort_values('rg_rec_med', ascending=False).head(5)

,cat_desc,macro_avg,num_instances,rg_rec_med,rg_prec_med,bs_rec_med,bs_prec_med
15,Physician :Physician Resident Admission Note,0.853431,13,0.998481,0.979633,0.992970,0.992560
6,Nursing:Nursing Transfer Note,0.590977,159,0.867188,0.757576,0.785322,0.744731
10,Physician :Physician Resident Progress Note,0.517666,34,0.734496,0.688121,0.896835,0.876756
19,Physician :Physician Surgical Progress Note,0.738827,7,0.717286,0.637016,0.883960,0.858172
7,Respiratory :Respiratory Care Shift Note,0.407171,79,0.698630,0.679104,0.751476,0.742840


## Analysis of ALL MIMIC

In [6]:
all_mimic_res = pd.concat([pd.read_pickle(f) for f in glob('rouge-and-bert-score-res-dfs/*.pickle')])

In [7]:
len(all_mimic_res.hadm_id.unique())

38685

In [8]:
all_mimic_avgs = compute_avgs(all_mimic_res)

In [10]:
def avg_over_toks(df):
    def avg_col(col):
        return (df.total_toks * df[col]).sum() / df.total_toks.sum()
    return avg_col('micro_avg'), avg_col('rg_rec_avg'), avg_col('rg_prec_avg'), \
        avg_col('bs_rec_avg'), avg_col('bs_prec_avg')

In [11]:
avg_over_toks(all_mimic_avgs)

(0.2636800382046393,
 0.42792174543186917,
 0.4233153323191832,
 0.584446257648712,
 0.5816599581880683)

In [12]:
avgs_condensed = all_mimic_avgs.loc[:, ['cat_desc', 'num_instances', 'macro_avg', 'rg_rec_med', 'rg_prec_med', 'bs_rec_med', 'bs_prec_med']]

In [13]:
cmap = sns.color_palette('flare', as_cmap=True)
cmap = sns.light_palette('cornflowerblue', as_cmap=True)

In [14]:
avgs_condensed.columns

Index(['cat_desc', 'num_instances', 'macro_avg', 'rg_rec_med', 'rg_prec_med',
       'bs_rec_med', 'bs_prec_med'],
      dtype='object')

In [15]:
mac_avg = avgs_condensed.loc[:, ['cat_desc', 'macro_avg']].sort_values('macro_avg', ascending=False).reset_index(drop=True)
rg = avgs_condensed.loc[:, ['cat_desc', 'rg_rec_med', 'rg_prec_med']].sort_values('rg_rec_med', ascending=False).reset_index(drop=True)
bs = avgs_condensed.loc[:, ['cat_desc', 'bs_rec_med', 'bs_prec_med']].sort_values('bs_rec_med', ascending=False).reset_index(drop=True)

In [22]:
df = avgs_and_format(all_mimic_res)

In [ ]:
df.loc[:, [('Type', ''), ('ROUGE', 'Recall')]]

In [53]:
def avgs_and_format(df, head=20):
    df = compute_avgs(df)
    df = df.loc[:, ['cat_desc', 'num_instances', 'macro_avg', 'rg_rec_med', 'rg_prec_med', 'bs_rec_med', 'bs_prec_med']]
    df.columns = [',Type', ',# Notes', ',DiffLib', 'ROUGE,Recall', 'ROUGE,Precision', 'BERTScore,Recall', 'BERTScore,Precision']
    df.columns = pd.MultiIndex.from_tuples(df.columns.str.split(',', expand=True).values)
    heading_properties = [('font-size', '12px'), ('text-align', 'center')]

    cell_properties = [('font-size', '13px')]

    dfstyle = [dict(selector="th", props=heading_properties),\
     dict(selector="td", props=cell_properties)]

#     df.style.set_table_attributes('style=font-size: 40px')
    df = df.loc[:, [('', 'Type'), ('', '# Notes'), ('', 'DiffLib'), ('ROUGE', 'Recall'), ('ROUGE', 'Precision'), ('BERTScore', 'Recall'), ('BERTScore', 'Precision')]]\
        .sort_values(('ROUGE', 'Recall'), ascending=False).reset_index(drop=True)\
        .head(20)
    return df, df.style.background_gradient(cmap, subset=[
        ('', 'DiffLib'), ('ROUGE', 'Recall'), ('ROUGE', 'Precision'), ('BERTScore', 'Recall'), ('BERTScore', 'Precision')], axis=0)\
        .set_table_styles(dfstyle)\
        .set_precision(2)

In [54]:
mcm_df, styled_df = avgs_and_format(all_mimic_res)

In [66]:
def weighted_avg(df):
    return ((df['# Notes'] / df['# Notes'].sum()) * df['DiffLib']).sum(),\
    ((df['# Notes'] / df['# Notes'].sum()) * df['RG R']).sum(),\
    ((df['# Notes'] / df['# Notes'].sum()) * df['RG P']).sum(),\
    ((df['# Notes'] / df['# Notes'].sum()) * df['BS R']).sum(),\
    ((df['# Notes'] / df['# Notes'].sum()) * df['BS P']).sum()

In [280]:
weighted_avg(mcm_df)

(0.21505814788701702,
 0.3030732784730055,
 0.29988579090196477,
 0.488720143482771,
 0.4869750266105248)

## KCH Stroke Notes Analysis

In [ ]:
import pandas as pd
from differ import compute_sequential_diff_metrics
kch_df = pd.read_pickle('stroke_notes.pickle')
kch_df['text'] = kch_df.body_analysed
sort_cols = ['clientvisit_guid', 'updatetime', 'document_description']
group_cols = ['clientvisit_guid', 'document_description']
kch_df_diffs = compute_sequential_diff_metrics(kch_df, sort_cols, group_cols) 

In [ ]:
def get_note_pairs(df, shuffled_visits, types_to_include, admission_col,  sort_cols, group_cols):
    note_pairs = defaultdict(list)
    for visit in shuffled_visits:
        if len(note_pairs) >= 10:
            break
        rows = df[df[admission_col] == visit].sort_values(sort_cols)
        for k, g in rows.groupby(group_cols):
            if g.shape[0] > 1 and k in types_to_include:
                for i in range(len(g.text.tolist())-1):
                    note_pairs[k].append((visit, g.text.iloc[i], g.text.iloc[i+1]))
    return note_pairs

In [209]:
shuffled_visits = pd.Series(kch_df.clientvisit_guid.unique()).sample(frac=1.0)
types_to_include = kch_avgs_df[('', 'Type')].tolist()
types_to_include = [t[0:-1] for t in types_to_include]

In [255]:
samples = get_note_pairs(kch_df, shuffled_visits, types_to_include, 'clientvisit_guid', ['updatetime', 'document_description'], 'document_description')

In [246]:
rouge_metric = load_metric('./rouge.py')
bert_metric = load_metric('./bertscore.py')
def output_note_pair(pair):
    print(pair[0])
    rouge_metric.add(prediction=pair[2], reference=pair[1])
    bert_metric.add(prediction=pair[2], reference=pair[1])
    rougeL = rouge_metric.compute(rouge_types=['rougeL'], use_agregator=False)['rougeL']
    bert_score = bert_metric.compute(lang='en', rescale_with_baseline=True, model_type='./xlnet-base-cased')
    print(f'Rouge Rec:{round(rougeL[0].recall, 3)}  Prec:{round(rougeL[0].precision, 3)}')
    print(f'BERTScore Rec:{round(float(bert_score["recall"].cpu().numpy()), 3)} BERTSCore Prec:{round(float(bert_score["precision"].cpu().numpy()), 3)}')
    show_diffs(pair[1], pair[2])

In [287]:
def review_pairs(samples):
    for note_type, pairs in samples.items():
        print('NOTE TYPE:' + str(note_type))
        input()
        for pair in pairs:
            clear_output(wait=True)
            output_note_pair(pair)
            skip_type = input()
            if skip_type == 'skip':
                break
        

### MIMIC FULL Manual Review

In [272]:
shuffled_visits = pd.Series(all_data.hadm_id.unique()).sample(frac=1.0)

In [273]:
types_to_include = [tuple(t.split(':')) for t in mcm_df[('', 'Type')]]

In [275]:
note_pairs = get_note_pairs(all_data, shuffled_visits, types_to_include, 'hadm_id', 
                            ['chartdate', 'charttime', 'category', 'description'],
                            ['category', 'description'])

In [277]:
[(k, len(v)) for k,v in note_pairs.items()]

[(('ECG', 'Report'), 8),
 (('Echo', 'Report'), 3),
 (('Nursing/other', 'Report'), 26),
 (('Radiology', 'CHEST (PORTABLE AP)'), 15),
 (('General', 'ICU Event Note'), 1),
 (('Nursing', 'Nursing Progress Note'), 10),
 (('Nursing', 'Nursing Transfer Note'), 1),
 (('Physician ', 'Physician Attending Progress Note'), 1),
 (('Physician ', 'Physician Resident Admission Note'), 1),
 (('Physician ', 'Physician Resident Progress Note'), 1)]

In [ ]:
review_pairs(note_pairs)

In [ ]:
list(sampled_notes.sort_values(sort_cols).groupby(group_cols))

In [75]:
kch_avgs_df, styled_df = avgs_and_format(kch_df_diffs)

In [63]:
avg_over_toks(compute_avgs(kch_df_diffs))

(0.24334423874924335,
 0.4169590003434991,
 0.4281630516124084,
 0.6039267108678945,
 0.6059166560277742)

In [ ]:
weighted_avg(kch_avgs_df)

In [ ]:
kch_avgs.loc[:, ['Type', '# Notes', 'DiffLib Avg', 'RG Rec', 'RG Prec', 'BS Rec', 'BS Prec']]\
    .sort_values('RG Rec', ascending=False).reset_index(drop=True)\
    .style.background_gradient(cmap, subset=['DiffLib Avg', 'RG Rec', 'RG Prec', 'BS Rec', 'BS Prec'], axis=0)\
    .set_precision(3)

In [ ]:
group_num = 1
rand_group = list(kch_df.groupby(['clientvisit_guid', 'document_description', 'updatetime']))[0][group_num]
print(rand_group['document_description'])
print(rand_group['text'].tolist()[0])

In [149]:
bert_metric = load_metric('./bertscore.py')

In [150]:
bert_metric.add(prediction='foo some text', reference='foo some other text ')

In [151]:
bert_metric.compute(lang='en', rescale_with_baseline=True, model_type='./xlnet-base-cased')

2021-03-01 08:20:18,052: Removing /home/thomass/.cache/huggingface/metrics/bert_score/default/default_experiment-ad0cea4d-c04d-4ec5-90ba-eb6d691977d4-1-0.arrow


{'precision': tensor([0.8856]),
 'recall': tensor([0.8320]),
 'f1': tensor([0.8588]),
 'hashcode': './xlnet-base-cased_L5_no-idf_version=0.3.7(hug_trans=4.3.0.dev0)-rescaled'}

### Randomly sample notes of interesting types
- rouge_recall of physicain: intesivist note and nursing transfer
- rouge_recall of 

In [ ]:
from IPython.display import clear_output

category = 'Physician '
description = 'Intensivist Note'
metric = 'rg_recall_med'
sample_size = 2

In [ ]:
random_sample = df[(df.category == category) & (df.description == description)].sort_values(sort_col, ascending=False).sample(sample_size).loc[:, [metric, 'hadm_id']]
for k, g in stroke_notes[stroke_notes.hadm_id.isin(random_sample.hadm_id) & (stroke_notes.description == description) & (stroke_notes.category == category)]\
        .sort_values(['hadm_id', 'category', 'description', 'charttime'])\
        .groupby(['hadm_id', 'category', 'description']):        
    for i in range(len(g.text) - 1):
        print(f'Group Key:{k}')
        print(f'Recall Median:{random_sample[random_sample.hadm_id == k[0]][metric].tolist()[0]}')
(g.text.iloc[i], g.text.iloc[i + 1])
        input('\n\nEnter for next...')
        clear_output()

In [28]:
avgs.loc[:, ['cat_desc', 'micro_avg', 'rg_recall_avg', 'rg_precision_avg', 'bs_recall_avg', 'bs_precision_avg']]

,cat_desc,micro_avg,rg_recall_avg,rg_precision_avg,bs_recall_avg,bs_precision_avg
0,Nursing/other:Report,0.071173,0.135753,0.137183,0.256340,0.255637
1,Radiology:CHEST (PORTABLE AP),0.173144,0.380864,0.382532,0.748738,0.746997
2,Nursing:Nursing Progress Note,0.266359,0.409362,0.399518,0.550573,0.543845
3,ECG:Report,0.317580,0.405494,0.408634,0.465594,0.471461
4,Radiology:CT HEAD W/O CONTRAST,0.139997,0.388552,0.430355,0.735919,0.744523
5,Physician :Intensivist Note,0.438114,0.717188,0.716120,0.886030,0.886067
6,Nursing:Nursing Transfer Note,0.521301,0.668460,0.668038,0.710947,0.712035
7,Respiratory :Respiratory Care Shift Note,0.428573,0.653369,0.639566,0.702126,0.696569
8,Echo:Report,0.199431,0.446879,0.460420,0.701508,0.717413
9,Radiology:MR HEAD W/O CONTRAST,0.179018,0.339165,0.740209,0.698005,0.808940


In [14]:
stroke_care_notes_df, stroke_discharge_df = split_notes(stroke_notes)

In [11]:
comp_metrics.sort_values('rougeL_rec', ascending=False)

,hadm_id,category,description,avg_diff_ratio,rougeL_rec,rougeL_prec
247,121824,Physician,Physician Resident Admission Note,0.853598,1.000000,0.769582
98,107913,Physician,Physician Surgical Admission Note,0.996592,1.000000,0.980357
860,180112,Nursing,Nursing Transfer Note,0.932375,1.000000,0.420744
450,140761,Physician,Physician Surgical Admission Note,0.999477,1.000000,0.996753
586,154473,General,Cardiology consult,0.904114,1.000000,0.949074
...,...,...,...,...,...,...
1079,195985,Nursing/other,Report,0.035890,0.073285,0.183257
889,182153,Nursing/other,Report,0.036163,0.066773,0.135223
799,171903,Nursing/other,Report,0.028571,0.060440,0.180328
122,109832,Nursing,Nursing Transfer Note,0.037115,0.049407,0.806452


In [30]:
texts = stroke_notes[(stroke_notes.hadm_id == 121824) & (stroke_notes.category == 'Physician ') & (stroke_notes.description == 'Physician Resident Admission Note')].text.tolist()

In [1]:
print("%10.3aE" % 2147889734063751200)

 2.148E+18


In [2]:
print("%10.3E" % 87008250040418300)

 8.701E+16


In [ ]:
print("%10.3E" %)

In [45]:
group_avgs = compute_avgs(results_df)

In [46]:
group_avgs['redundant_ratio'] = group_avgs['redundant_toks'] / group_avgs.total_toks

In [47]:
group_avgs.sort_values('redundant_ratio', ascending=False).reset_index(drop=True).head(20)

,cat_desc,redundant_toks,total_toks,avg_txt_len,macro_avg,micro_avg,num_instances,redundant_ratio
0,Physician :Physician Resident Admission Note,28842.0,33109,12525.819048,0.856518,0.809448,19,0.871123
1,Physician :Physician Surgical Progress Note,3181.0,4347,5587.000000,0.809635,0.764214,7,0.731769
2,Physician :Physician Attending Progress Note,14380.0,21148,5941.186706,0.532457,0.536417,31,0.679970
3,Physician :Intensivist Note,100737.0,159909,5958.690231,0.691003,0.639584,211,0.629965
4,Respiratory :Respiratory Care Shift Note,7263.0,12124,1230.545915,0.404803,0.443782,79,0.599060
5,Nursing:Nursing Transfer Note,30799.0,53260,2230.841242,0.598621,0.550831,159,0.578276
6,Physician :Physician Resident Progress Note,40675.0,73517,6890.886364,0.551072,0.467104,94,0.553273
7,Nursing:Nursing Progress Note,64447.0,136297,1291.039215,0.442277,0.419754,620,0.472842
8,Radiology:PORTABLE ABDOMEN,923.0,2992,1084.420513,0.274080,0.259291,23,0.308489
9,Radiology:CHEST PORT. LINE PLACEMENT,1237.0,4903,1240.143860,0.203611,0.219498,34,0.252295


In [91]:
pt = stroke_notes[stroke_notes.hadm_id == 195402]

2021-01-07 10:30:29,573: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow


In [100]:
pt_nurs_notes = pt[pt.description == 'Nursing Progress Note']

In [125]:
def single_group_ratios(pt):
    internote_sort_cols = ['hadm_id', 'chartdate', 'charttime', 'category', 'description']
    internote_group_keys = ['hadm_id', 'category', 'description']
    groups_gen = pt.sort_values(internote_sort_cols).groupby(internote_group_keys)
    results = defaultdict(list)
    metric = load_metric('rouge')
    score = None
    ratios = []
    for k, g in [list(groups_gen)[-1]]:
        if g.shape[0] > 1:
            for i in range(len(g.text.tolist()) - 1):
                metric.add(prediction=g.text.iloc[i], reference=g.text.iloc[i+1])
                ratio, std_dev, max_ratio, min_ratio, redundant_toks_len, total_toks = \
                    diff_ratio(g.text.iloc[i], g.text.iloc[i+1])
                ratios.append(ratio)
            score = metric.compute(rouge_types=['rougeL'])
    return ratios, score

In [126]:
ratios, rouge_scores = single_group_ratios(pt_nurs_notes)

2021-01-07 11:32:03,220: Removing /Users/tom/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow
